In [3]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 69.0 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3


Model page: https://huggingface.co/meta-llama/Llama-4-Maverick-17B-128E-Instruct

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/meta-llama/Llama-4-Maverick-17B-128E-Instruct)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

The model you are trying to use is gated. Please make sure you have access to it by visiting the model page.To run inference, either set HF_TOKEN in your environment variables or Secrets or run the following cell to login. 🤗

In [18]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer = T5Tokenizer.from_pretrained("t5-base")

2025-05-30 06:14:08.911260: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748585649.115453      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748585649.174649      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [5]:
input_text=input("Enter a paragraph :")

KeyboardInterrupt: Interrupted by user

In [ ]:
inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

outputs = model.generate(inputs, max_length=15, num_beams=4, early_stopping=True)
headline = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Generated Headline:", headline)


## Fine tuning "T5 base" with a dataset

In [11]:
from datasets import load_dataset, Dataset
import pandas as pd

In [12]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("banuprakashv/news-articles-classification-dataset-for-nlp-and-ml")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/news-articles-classification-dataset-for-nlp-and-ml


In [13]:
df = pd.read_csv("/kaggle/input/news-articles-classification-dataset-for-nlp-and-ml/business_data.csv")  # columns: article, headline
df = df.dropna()
df["input_text"] = "headlines: " +df['description']+ df["content"]
df["target_text"] = df["headlines"]

In [14]:
dataset = Dataset.from_pandas(df[["input_text", "target_text"]])
dataset = dataset.train_test_split(test_size=0.1)

In [15]:
import re

def clean_text(text):
    # Example: remove newlines and extra spaces
    text = re.sub(r'\n', ' ', text)          # Replace newlines with space
    text = re.sub(r'\s+', ' ', text).strip()  # Replace multiple spaces with one and trim
    return text

In [16]:
def preprocess(examples):
    inputs = tokenizer(examples["input_text"], truncation=True, padding="max_length", max_length=512)
    targets = tokenizer(examples["target_text"], truncation=True, padding="max_length", max_length=32)
    inputs["labels"] = targets["input_ids"]
    return inputs

In [19]:
tokenized_datasets = dataset.map(preprocess, batched=True)

Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [20]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_text', 'target_text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1800
    })
    test: Dataset({
        features: ['input_text', 'target_text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 200
    })
})

In [14]:
from transformers import TrainingArguments,Trainer

In [15]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [16]:
training_args = TrainingArguments(
    output_dir="./t5-headline-generator",
    report_to="none",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

In [17]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
500,1.717900


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=675, training_loss=1.6283154748987267, metrics={'train_runtime': 554.9775, 'train_samples_per_second': 9.73, 'train_steps_per_second': 1.216, 'total_flos': 3288372609024000.0, 'train_loss': 1.6283154748987267, 'epoch': 3.0})

In [21]:
model.save_pretrained("headline-t5-model")
tokenizer.save_pretrained("headline-t5-model")

('headline-t5-model/tokenizer_config.json',
 'headline-t5-model/special_tokens_map.json',
 'headline-t5-model/spiece.model',
 'headline-t5-model/added_tokens.json')

In [50]:
text=input("enter the content :")

enter the content : Chandigarh: At least four migrant workers killed and several others were injured in a blast in a firecracker manufacturing and packaging unit near a village in Punjab's Sri Muktsar Sahib district on Friday, police said.   SSP of Sri Muktsar Sahib, Akhil Chaudhary said, "...four people died when the building collapsed following the explosion. Rescue operations are underway and the injured have been admitted to the hospital..."  Jaspal Singh, Deputy Superintendent of Police (DSP), Lambi said, "Late last night, a blast occurred at a firecracker factory...Almost 50 labourers work in the factory...Four bodies have been recovered and 27 injured have been admitted to the hospital."  The incident was reported in the two-storey factory unit located on Singhawali-Kotli road in Sri Muktsar Sahib, Lambi's Deputy Superintendent of Police, Jaspal Singh, said over the phone.  However, the cause of the blast is under investigation, the DSP said.  More details to be added soon....


In [53]:
model = T5ForConditionalGeneration.from_pretrained("headline-t5-model")
tokenizer = T5Tokenizer.from_pretrained("headline-t5-model")

inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

outputs = model.generate(**inputs, max_length=41)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


four people killed and 27 injured in blast at firecracker factory in Punjab . blast happened at two-storey unit near village in Sri Muktsar Sahib district


# with BART

In [21]:
df = pd.read_csv("/kaggle/input/news-articles-classification-dataset-for-nlp-and-ml/business_data.csv")  # columns: article, headline
df = df.dropna()
df["input_text"] = "headlines: " +df['description']+ df["content"]
df["target_text"] = df["headlines"]

In [22]:
import re

def clean_text(text):
    text = re.sub(r'\n', ' ', text)          
    text = re.sub(r'\s+', ' ', text).strip() 
    return text

def preprocess_function(examples):
    inputs = [clean_text(text) for text in examples['content']]
    targets = [clean_text(text) for text in examples['headlines']]
    
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=64, truncation=True, padding='max_length')
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs



In [23]:
from datasets import load_dataset

# Load your single CSV dataset
dataset = load_dataset('csv', data_files='/kaggle/input/news-articles-classification-dataset-for-nlp-and-ml/business_data.csv')

# Split into train and validation (e.g., 80% train, 20% val)
split_datasets = dataset['train'].train_test_split(test_size=0.2)

# Access splits
train_dataset = split_datasets['train']
val_dataset = split_datasets['test']

# Apply preprocessing on both splits
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [24]:
from transformers import BartTokenizer, BartForConditionalGeneration

tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [25]:
import os 
os.environ["WANDB_DISABLED"] = "true"

In [26]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./bart-headline-results',
    report_to="none",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    logging_dir='./logs',
)


In [27]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
)


In [28]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
500,1.883700


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3464: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


KeyboardInterrupt: 

In [1]:
import requests

# URL of a public API
url = "https://jsonplaceholder.typicode.com/posts/1"

# Send GET request
response = requests.get(url)

# Print the status code
print("Status Code:", response.status_code)

# Print the response data (JSON)
print("Response JSON:", response.json())


Status Code: 200
Response JSON: {'userId': 1, 'id': 1, 'title': 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit', 'body': 'quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto'}
